# Neural Networks from scratch in Python 

In [2]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

### Dense Layer

In [3]:
class Layer_Dense:

    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases  = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        # Gradient on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases  = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs  = np.dot(dvalues, self.weights.T)

### ReLU activation

In [4]:
class Activation_ReLU:

    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs

        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Since we need to modify the original variable, 
        # let's make a copy of the variable first 
        self.dinputs = dvalues.copy()

        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

### Softmax activation

In [5]:
class Activation_Softmax:

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs

        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):

        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1,1)
            # Calculate Jacobian matrix of the output ...
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)

            # Calculate sample-wise gradient and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

### Common loss class

In [6]:
class Loss:

    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):

        # Calculate sample losses
        sample_losses = self.forward(output, y)

        # Calculate mean loss
        return np.mean(sample_losses)

    def forward(self, y_pred, y_true):
        return []

### Cross-entropy loss (subclass to Loss)

In [7]:
class Loss_CategoricalCrossentropy(Loss):

    # Forwad pass
    def forward(self, y_pred, y_true):

        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        # Probabilities for target values - 
        # only if categorical labels
        # correct_confidences = 0  # Remove warning!
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]

        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        
        #else:
        #    print("ERROR!")
        #    correct_confidences = 0

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample.
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector 
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        # Calculate gradient
        self.dinputs = -y_true / dvalues

        # Normalize gradient
        self.dinputs = self.dinputs / samples

### Activation_Softmax_Loss_CategoricalCrossEntropy

In [8]:
# softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy():

    # Creates activation and loss funtion objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss       = Loss_CategoricalCrossentropy()

    # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        
        # If labels are one-hot encoded, turn them into discrete values
        if len(y_true.shape) == 2: 
            y_true = np.argmax(y_true, axis=1)
            
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples

### Optimizer SGD (Stochastic Gradient Descent)

In [15]:
class Optimizer_SGD:

    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer

    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.learning_rate          = learning_rate 
        self.current_learning_rate  = learning_rate
        self.decay                  = decay
        self.iterations             = 0
        self.momentum               = momentum
    
    # Call once before any parameters update
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations)) 

    def update_params(self, layer):
        if self.momentum:

            # If layer does not contain momentum arrays, create 
            # them filled with zeroes (weight and biases)
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums   = np.zeros_like(layer.biases)

            # Build weight updates with momentum - take previous 
            # updates multiplied by retain factor and update with
            # current gradients
            weight_updates = \
                self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weights_momentums = weight_updates

            # .. and build biases updates 
            bias_updates = \
                self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentum = bias_updates

        else:   
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates   = -self.current_learning_rate * layer.dbiases
        
        layer.weights += weight_updates
        layer.biases  += bias_updates
        
    # Call once after any parameters updates
    def post_update_params(self):
        self.iterations += 1

### Run The model

In [16]:
# Create Dataset
X, y = spiral_data(samples=100, classes=3)

# Create the network model. 
dense1      = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2      = Layer_Dense(64,3)

# Create Softmax combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(learning_rate=1.0, decay=1e-3, momentum=0.9)

# Train in loop
for epoch in range(10001):

    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)

    # Loss 
    loss = loss_activation.forward(dense2.output, y)
    
    # Calculate accuracy. Calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2: 
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100: 
        print(f"epoch: {epoch:05d} | acc: {accuracy:.3f} | loss: {loss:.3f} | lr: {optimizer.current_learning_rate}")

    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()
    

epoch: 00000 | acc: 0.353 | loss: 1.099 | lr: 1.0
epoch: 00100 | acc: 0.467 | loss: 1.079 | lr: 0.9099181073703367
epoch: 00200 | acc: 0.450 | loss: 1.067 | lr: 0.8340283569641367
epoch: 00300 | acc: 0.447 | loss: 1.065 | lr: 0.7698229407236336
epoch: 00400 | acc: 0.437 | loss: 1.064 | lr: 0.7147962830593281
epoch: 00500 | acc: 0.430 | loss: 1.062 | lr: 0.66711140760507
epoch: 00600 | acc: 0.427 | loss: 1.061 | lr: 0.6253908692933083
epoch: 00700 | acc: 0.433 | loss: 1.059 | lr: 0.5885815185403178
epoch: 00800 | acc: 0.453 | loss: 1.056 | lr: 0.5558643690939411
epoch: 00900 | acc: 0.467 | loss: 1.054 | lr: 0.526592943654555
epoch: 01000 | acc: 0.467 | loss: 1.051 | lr: 0.5002501250625312
epoch: 01100 | acc: 0.463 | loss: 1.048 | lr: 0.4764173415912339
epoch: 01200 | acc: 0.463 | loss: 1.045 | lr: 0.45475216007276037
epoch: 01300 | acc: 0.467 | loss: 1.041 | lr: 0.43497172683775553
epoch: 01400 | acc: 0.470 | loss: 1.037 | lr: 0.4168403501458941
epoch: 01500 | acc: 0.480 | loss: 1.032 |

In [11]:
# Test - decaying learning rate
starting_learning_rate = 1.0
learning_rate_decay    = 0.1

for step in range(20):
    learning_rate = starting_learning_rate * (1 /(1+learning_rate_decay*step))
    print(step, f"{learning_rate:.3f}")

0 1.000
1 0.909
2 0.833
3 0.769
4 0.714
5 0.667
6 0.625
7 0.588
8 0.556
9 0.526
10 0.500
11 0.476
12 0.455
13 0.435
14 0.417
15 0.400
16 0.385
17 0.370
18 0.357
19 0.345
